<h1><center>  企业信用调查 -  Nucleus API实例</center></h1>


<h1><center>  所有权及保密条款属SumUp Analytics所有</center></h1>
<h1><center>  免责声明和服务条款可通过www.sumup.ai获取</center></h1>


#  

 


## 目标: 
-	使用公司发布的内容构建公司债券排名


## 数据:
-	选定的公司名单（可能属于同一行业或具有类似的市值）
    
- 	公司报告（例如，SEC备案）
- 	新闻稿
- 	财报电话会议记录



## Nucleus APIs:
-	数据集创建 API
 - 	*api_instance.post_upload_file(file, dataset)*
 - 	*nucleus_helper.import_files(api_instance, dataset, file_iters, processes=1)*

        nucleus_helper.import_files利用api_instance.post_upload_file并行执行来加速数据集的创建


-	主题建模 API
 - 	*api_instance.post_topic_api(payload)*


-	主题情绪 API
 - 	*api_instance.post_topic_sentiment_api(payload)*


-	DocInfo API
 - 	*api_instance.post_doc_info(payload)*


-	DatasetInfo API
 - 	*api_instance.post_dataset_info(payload)*


## 方法:

### 1.	数据集准备
-	创建一个Nucleus数据集，其中包含所选历史时期内的所有相关文档

    

In [ ]:
import csv
import json
import nucleus_api.api.nucleus_api as nucleus_helper
import nucleus_api
from nucleus_api.rest import ApiException

configuration = nucleus_api.Configuration()
configuration.host = 'UPDATE-WITH-API-SERVER-HOSTNAME'
configuration.api_key['x-api-key'] = 'UPDATE-WITH-API-KEY'

# 创建API实例
api_instance = nucleus_api.NucleusApi(nucleus_api.ApiClient(configuration))

In [ ]:
print('--------- Append all files from local folder to dataset in parallel -----------')
folder = 'Corporate_documents'         
dataset = 'Corporate_docs'# str | 将插入文件的目标数据集。

# 以递归方式从文件夹构建文件。
# 每个项目采用以下格式：
# {'filename': filename,   # 要上传的文件名。 需要
#  'metadata': {           # 该文件的元数据。 可选的
#      'key1': val1,       # 只要名称，密钥就可以有任意名称
#      'key2': val2        # 包含字母数字（0-9 | a-z | A-Z）和下划线（_）
#   } 
# }
file_iter = []
for root, dirs, files in os.walk(folder):
    for file in files:
        #if Path(file).suffix == '.pdf': # .txt .doc .docx .rtf .html .csv also supported
            file_dict = {'filename': os.path.join(root, file),
                         'metadata': {'ticker': 'AAPL',
                                      'company': 'Apple',
                                      'category': 'Press Release',
                                      'date': '2019-01-01'}}
            file_iter.append(file_dict)

file_props = nucleus_helper.upload_files(api_instance, dataset, file_iter, processes=4)
for fp in file_props:
    print(fp.filename, '(', fp.size, 'bytes) has been added to dataset', dataset)

-	使用SEC文件和Nucleus自带数据传输的替代方案

In [ ]:
dataset = "Corporate_docs" 
period_start = "2010-01-01" 
period_end= "2019-06-01"

payload = nucleus_api.EdgarQuery(destination_dataset=dataset,
                                tickers=["FB", "AMZN", "INTL", "IBM", "NFLX", "GOOG"], 
                                filing_types=["10-K", "10-K/A", "10-Q", "10-Q/A"], 
                                sections=["Quantitative and Qualitative Disclosures about Market Risk",
                                          "Management's Discussion and Analysis of Financial Condition and Results of Operations",
                                          "Risk Factors"],
                                period_start=period_start,
                                period_end=period_end)

api_response = api_instance.post_create_dataset_from_sec_filings(payload)

**您可以直接在API中处理数据集中的特定时间段，如下所示**

### 2.	情绪和主题贡献（筛选分析）
- 在给定时间识别和提取文档子集中的关键主题


- 衡量每个主题的情绪，将所有关键主题分类为“好”和“坏”主题


- 确定每个公司对每个主题的曝光度


- 根据主题的“好”或“坏”性质，将特定公司的风险总结在关键主题上，以得出公司的排名
 - 顶级公司是最能接触好主题和/或最少接触不良主题的公司
 
 
- 接下来，我们将讨论如何通过利用用户可用的不同参数来优化此分析




In [ ]:
# 确定哪些公司与主题相关的文档相关联
import numpy as np

payload = nucleus_api.DocInfo(dataset='Corporate_docs')
api_response = api_instance.post_doc_info(payload)

company_sources = []
for res in api_response.result:        
    company_sources.append(res.attribute['ticker']) 

company_list = np.unique(company_sources)


print('-------- Get topic sentiment and exposure per firm ----------------')

payload = nucleus_api.TopicSentimentModel(dataset='Corporate_docs',          
                                        query='',                   
                                        num_topics=20,
                                        num_keywords=8,
                                        period_start="2018-11-01 00:00:00",
                                        period_end="2019-01-01 00:00:00")
try:
    api_response = api_instance.post_topic_sentiment_api(payload)    
    api_ok = True
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])
    api_ok = False

if api_ok:   
    company_rankings = np.zeros([len(company_list), len(api_response.result)])
    for i, res in enumerate(api_response.result):
        print('Topic', i, 'sentiment:')
        print('    Keywords:', res.keywords)

        # 使用数据集metadata将主题中的所有文档曝光聚合到公司曝光中
        payload = nucleus_api.DocInfo(dataset='Corporate_docs', doc_ids = res.doc_ids)
        api_response1 = api_instance.post_doc_info(payload)

        company_sources = [] # 此列表将比整个数据集短得多，因为并非所有文档都对特定主题有贡献
        for res1 in api_response1.result:        
            company_sources.append(res1.attribute['ticker']) 

        company_contributions = np.zeros([len(company_list), 1])
        for j in range(len(company_list)):
            for k in range(len(company_sources)):
                if company_sources[k] == company_list[j]:
                    company_contributions[j] += json.loads(res.doc_topic_exposures[0])[k]

        company_rankings[:, i] = [x[0] for x in  float(res.strength) * float(res.sentiment) * company_contributions[:]]  

        print('---------------')


    # 将每个主题的公司排名添加到最终信用筛选中
    Corporate_screen = np.mean(company_rankings, axis=1)

-	对历史期间的每个日期重复上述任务，以获取信用屏幕的完整历史记录

In [ ]:
import datetime
import numpy as np

print('------------ Retrieve all companies found in the dataset ----------')

payload = nucleus_api.DocInfo(dataset='Corporate_docs')
api_response = api_instance.post_doc_info(payload)

company_sources = []
for res in api_response.result:        
    company_sources.append(res.attribute['ticker']) 

company_list = np.unique(company_sources)


print('--------------- Retrieve the time range of the dataset -------------')

payload = nucleus_api.DatasetInfo(dataset='Corporate_docs', query='')
api_response = api_instance.post_dataset_info(payload)

first_date = datetime.datetime.fromtimestamp(float(api_response.result.time_range[0]))
last_date = datetime.datetime.fromtimestamp(float(api_response.result.time_range[1]))
delta = last_date - first_date

# 遍历时间和每个日期，计算公司的排名
T = 90 # The look-back period in days

Corporate_screen = []
for i in range(delta.days):  
    if i == 0:
        end_date = first_date + datetime.timedelta(days=T)
 
    # 回顾T周期的第一个和最后一个日期
    start_date = end_date - datetime.timedelta(days=T)
    start_date_str = start_date.strftime("%Y-%m-%d 00:00:00")

    # 我们想要一个每日指标
    end_date = end_date + datetime.timedelta(days=1) 
    end_date_str = end_date.strftime("%Y-%m-%d 00:00:00")

    payload = nucleus_api.TopicSentimentModel(dataset="Corporate_docs",      
                                            query='',                   
                                            num_topics=20,
                                            num_keywords=8,
                                            period_start=start_date_str,
                                            period_end=end_date_str)
    try:
        api_response = api_instance.post_topic_sentiment_api(payload)
        api_ok = True
    except ApiException as e:
        api_error = json.loads(e.body)
        print('ERROR:', api_error['message'])
        api_ok = False

    if api_ok:   
        company_rankings = np.zeros([len(company_list), len(api_response.result)])
        for l, res in enumerate(api_response.result):
            # 使用数据集元数据将主题中的所有文档曝光聚合到公司曝光中
            payload = nucleus_api.DocInfo(dataset='Corporate_docs', doc_ids=res.doc_ids)
            api_response1 = api_instance.post_doc_info(payload)

            company_sources = [] # 此列表将比整个数据集短得多，因为并非所有文档都对特定主题有贡献
            for res1 in api_response1.result:        
                company_sources.append(res1.attribute['ticker']) 

            company_contributions = np.zeros([len(company_list), 1])
            for j in range(len(company_list)):
                for k in range(len(company_sources)):
                    if company_sources[k] == company_list[j]:
                        company_contributions[j] += json.loads(res.doc_topic_exposures[0])[k]

            company_rankings[:, l] = [x[0] for x in  float(res.strength) * float(res.sentiment) * company_contributions[:]]      

        # 将每个主题的公司排名添加到最终信用筛选中
        Corporate_screen.append(np.mean(company_rankings, axis=1))

### 3.	结果解释
-	绘制公司排名的时间序列与经过beta调整的公司债券利差

### 4.	微调

#### a.	量身定制主题

-   排除不相关的关键字/主题，以便使用Topic Sentiment API中的`custom stop_words`参数来定制公司屏幕


-	提取文档子集的关键主题并打印其关键字



In [ ]:
print('------------- Get list of topics from dataset --------------')

payload = nucleus_api.Topics(dataset='Corporate_docs',                       
                            query='',                       
                            num_topics=20, 
                            num_keywords=8,
                            period_start="2018-11-01 00:00:00",
                            period_end="2019-01-01 00:00:00")
try:
    api_response = api_instance.post_topic_api(payload)        
    api_ok = True
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])
    api_ok = False

if api_ok:       
    for i, res in enumerate(api_response.result.topics):
        print('Topic', i, ' keywords: ', res.keywords)    
        print('---------------')

您可以通过创建`custom_stop_words`变量来定制筛选分析。 如下所示的初始化变量，将其传递到第2节主代码的有效负载中：

In [ ]:
custom_stop_words = ["call","report"] # str | 停用词列表。 （可选）

#### b.	将屏幕分析集中在某些主题上
如果您决定关注屏幕分析，例如财务健康和公司行为主题，只需将第2节主要代码中的查询变量替换为：

In [ ]:
query = '(earnings OR debt OR competition OR lawsuit OR restructuring)' # str | 全文查询，使用mysql MATCH boolean query格式。 示例：“（word1 OR word2）AND（word3 OR word4）”（可选）

#### c.	探索文档类型，回顾期，提取的主题数量的影响
**num_topics**: 您可以通过更改第2节主代码中有效负载中的变量`num_topics`来使用不同广度的主题来计算公司筛选。
- 在建立排名时，较大的值将包含更广泛的主题/内容，而较小的值将提供具有主要主题的简明度量。
- 如果`num_topics`太大，一些非常边缘的话题可能会在衡量公司排名时带来很多噪音。

**T**: 您可以通过更改第2节主代码中的变量`T`（lookback）来计算具有不同传播速度的公司筛选。
- 较大的值将提供缓慢变化的排名，而较小的值将导致非常敏感的排名。
- 如果`T`太小，将只使用很少的文件，这可能会导致排名公司的噪音很大。
- 如果“T”太大，由于平滑效应，排名不会反映信息。

**Document types**: 与捕获整个数据集相比，您可以调查公司屏幕如何随不同类型的文档（例如公司报告，新闻稿或财报电话会议）而变化。 在整个语料库的子集上重新运行第2节的主要代码。 创建一个变量`metadata_selection`并将其传递给有效负载：

In [1]:
metadata_selection = {"category": "Report"}   # str | json object of {\"metadata_field\":[\"selected_values\"]} (optional)

### 5.	下一步
-  针对不同的行业部门重复上述任务

 - 这为您提供了每个行业的广泛市场筛选，您可以使用它来进行安全选择
 
 - 通过在所有行业的公司中使用单个数据集，您可以使用附加到公司/文档的行业部门的额外元数据参数来获得行业排名。 然后，您可以将此行业屏幕应用于行业分配


-  在公司排名的时间序列和他们的β调整后的信用差价之间进行相关性分析
 - 可以研究价格影响的几个时间范围：1天，7天，几周，或甚至更持久的影响
 
 - 可以研究价格影响的几个时间滞后：在市场开始调整之前，在市场开始整合来自公司内容的信息之前，在市场开始调整之前存在2至3天的差距，一周或甚至更长的差距
 
 - 您还可以通过在债券范围内创建一系列信用差价来进行此类相关性分析。 交易速度/频率与价格影响之间是否存在相关性？


-	探索指标的简单转换
 - 您可以使用横截面分数重新调整和平滑这些排名

        Score(Company i) = ( Rank(Company i) – Average(Ranks, [Companies]) ) / Std(Ranks, [Companies])

版权(c) 2019年 SumUp Analytics 公司 版权所有。 

通知：所有信息均属于SumUp Analytics Inc公司及其供应商的财产。 本合同所包含的知识产权和技术概念属于SumUp Analytics Inc.及其供应商的专利，可由美国和外国专利、在工艺中的专利以及受贸易秘密或版权法保护的专利涵盖。 

除非得到SumUp Analytics公司的事先书面批准，否则严禁传播此类信息或复制此材料。 